# Setup

In [1]:
import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass

In [2]:
#prompt user to enter MySQL root password
sql_pass = getpass.getpass()
#create connection string and engine to connect to MySQL database
connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/civictech'
engine = create_engine(connection_string)

In [3]:
import pandas as pd

# People

### Names

In [4]:
names= pd.read_csv('profiles/names.csv', sep=';')

In [5]:
names.to_sql('names', engine, 'civictech', if_exists='replace', index=False)

47

### Profiles

In [6]:
people=pd.read_csv('profiles/people_main_db.csv', sep=';')
print(list(people.columns))
print(len(list(people.columns)))

['ind_id', 'geoCountryName', 'geoLocationName', 'summary', 'industryName', 'headline', 'experience', 'education', 'languages', 'publications', 'certifications', 'volunteer', 'honors', 'projects', 'experience1', 'experience2', 'experience3', 'experience4', 'experience5', 'education1', 'education2', 'education3', 'languages_over2', 'honors_stated', 'publications_stated', 'volunteer_stated', 'projects_stated', 'certifications_stated', 'languages_stated', 'consulting_roles', 'direction_roles', 'founding_roles']
32


In [7]:
# We may want to drop the columns with too many missing values.
pd.DataFrame(people.isna().sum().sort_values(ascending=False)).head(15)

,0
summary,15
education3,12
education2,6
geoLocationName,4
experience5,3
experience4,2
geoCountryName,2
education1,1
honors_stated,0
languages_over2,0


In [8]:
# and/or drop text columns 
people.dtypes.sort_values()

ind_id                    int64
consulting_roles          int64
languages_stated          int64
certifications_stated     int64
projects_stated           int64
volunteer_stated          int64
publications_stated       int64
honors_stated             int64
languages_over2           int64
direction_roles           int64
founding_roles            int64
volunteer                object
geoCountryName           object
geoLocationName          object
summary                  object
industryName             object
headline                 object
experience               object
education                object
languages                object
certifications           object
education3               object
education1               object
experience5              object
experience4              object
experience3              object
publications             object
experience1              object
projects                 object
honors                   object
education2               object
experien

In [9]:
# for now we are keeping it as is
people.to_sql('people', engine, 'civictech', if_exists='replace', index=False)

47

### Experience

In [71]:
experience=pd.read_csv('profiles/people_experience_db.csv', sep=';')

In [72]:
print(list(experience.columns))
print(len(list(experience.columns)))
print(len(experience))

['ind_id', 'locationName', 'companyName', 'description', 'company', 'title', 'startDate_month', 'startDate_year', 'endDate_month', 'endDate_year', 'industry', 'company_empl_low', 'company_empl_high', 'title_direction', 'titleconsulting', 'titlefounder']
16
235


In [73]:
experience.isna().sum().sort_values(ascending=False)

endDate_month        82
endDate_year         79
locationName         62
description          57
company              33
company_empl_low     25
company_empl_high    25
industry             20
startDate_month      12
companyName           5
title                 5
startDate_year        5
ind_id                0
title_direction       0
titleconsulting       0
titlefounder          0
dtype: int64

In [74]:
experience.isna().sum().sort_values(ascending=False)

endDate_month        82
endDate_year         79
locationName         62
description          57
company              33
company_empl_low     25
company_empl_high    25
industry             20
startDate_month      12
companyName           5
title                 5
startDate_year        5
ind_id                0
title_direction       0
titleconsulting       0
titlefounder          0
dtype: int64

In [75]:
experience['startDate_year'] = experience['startDate_year'].fillna(0)
experience['startDate_month'] = experience['startDate_month'].fillna(0)
experience['endDate_year'] = experience['endDate_year'].fillna(0)
experience['endDate_month'] = experience['endDate_month'].fillna(0)
experience['company_empl_low'] = experience['company_empl_low'].fillna(999)
experience['company_empl_high'] = experience['company_empl_high'].fillna(999)

In [76]:
experience['startDate_year'] = experience['startDate_year'].astype(int)
experience['startDate_month'] = experience['startDate_month'].astype(int)
experience['endDate_year'] = experience['endDate_year'].astype(int)
experience['endDate_month'] = experience['endDate_month'].astype(int)
experience['company_empl_low'] = experience['company_empl_low'].astype(int)
experience['company_empl_high'] = experience['company_empl_high'].astype(int)

In [77]:
experience.dtypes.sort_values()

ind_id                int64
startDate_month       int64
startDate_year        int64
endDate_month         int64
endDate_year          int64
company_empl_low      int64
company_empl_high     int64
title_direction       int64
titleconsulting       int64
titlefounder          int64
locationName         object
companyName          object
description          object
company              object
title                object
industry             object
dtype: object

In [78]:
experience.drop(columns='company', inplace=True)

In [79]:
experience.industry[1]

"['Management Consulting']"

In [80]:
import re
def clean_industry(x):
    x=str(x)
    pattern = "[a-zA-Z]+ [a-zA-Z]+|[a-zA-Z]+"
    return re.findall(pattern, x)

In [81]:
experience['industry']= experience['industry'].apply(clean_industry)

In [82]:
experience['industry'].value_counts()

[Computer Software]                       55
[Information Technology, and Services]    45
[Civic, Social Organization]              35
[Management Consulting]                   20
[nan]                                     20
[Higher Education]                        20
[Internet]                                10
[Venture Capital, Private Equity]          5
[Nonprofit Organization, Management]       5
[Graphic Design]                           5
[Performing Arts]                          5
[Marketing and, Advertising]               5
[Research]                                 5
Name: industry, dtype: int64

In [83]:
experience.reset_index(inplace=True)
experience.rename(columns={"index":"exp_id"}, inplace=True)

In [84]:
experience.dtypes

exp_id                int64
ind_id                int64
locationName         object
companyName          object
description          object
title                object
startDate_month       int64
startDate_year        int64
endDate_month         int64
endDate_year          int64
industry             object
company_empl_low      int64
company_empl_high     int64
title_direction       int64
titleconsulting       int64
titlefounder          int64
dtype: object

In [85]:
experience.columns

Index(['exp_id', 'ind_id', 'locationName', 'companyName', 'description',
       'title', 'startDate_month', 'startDate_year', 'endDate_month',
       'endDate_year', 'industry', 'company_empl_low', 'company_empl_high',
       'title_direction', 'titleconsulting', 'titlefounder'],
      dtype='object')

In [87]:
experience.head()

,exp_id,ind_id,locationName,companyName,description,title,startDate_month,startDate_year,endDate_month,endDate_year,industry,company_empl_low,company_empl_high,title_direction,titleconsulting,titlefounder
0,0,0,Paris Metropolitan Region,bluenove,Bluenove accompagne la transformation positive...,Directeur associé,11,2017,0,0,[Management Consulting],11,50,1,0,0
1,1,0,Paris Metropolitan Region,démocratie ouverte,Démocratie Ouverte est un collectif citoyen in...,Co-Président,1,2018,5,2020,[Management Consulting],11,50,1,0,0
2,2,0,Paris Metropolitan Region,apm - association progrès du management,NaN,Expert,1,2017,0,0,[Management Consulting],11,50,0,0,0
3,3,0,NaN,démocratie ouverte,NaN,Membre du Comité d'Orientation Stratégique,7,2021,0,0,[Management Consulting],11,50,0,0,0
4,4,0,Paris Metropolitan Region,dassault systèmes,Netvibes provides Dashboard Intelligence ~ tra...,"Senior Director, Strategic Business Development",2,2013,10,2017,[Management Consulting],11,50,1,0,0


In [91]:
#editor.unusualLineTerminators: "auto"

#   // Remove unusual line terminators that might cause problems.
#   //  - auto: Unusual line terminators are automatically removed.
#   //  - off: Unusual line terminators are ignored.
#   //  - prompt: Unusual line terminators prompt to be removed.
#   "editor.unusualLineTerminators": "prompt",

NameError: name 'editor' is not defined

In [92]:
experience.to_csv("profiles/people_experience_db_2.csv", sep=";", index=False)

In [ ]:
from sqlalchemy.types import Integer, BigInteger, Boolean, Date, Float, LargeBinary, SmallInteger, String, Text, Unicode, UnicodeText

experience.to_sql('experience', con=engine, if_exists='replace', index=False,
          dtype={"exp_id": Integer(), 'ind_id': Integer(), 'startDate_month': Integer(), 'startDate_year': Integer(), 'endDate_month': Integer(),
       'endDate_year': Integer(), 'company_empl_low':Integer(), 'company_empl_high':Integer(), 'title_direction': Integer(), 'titleconsulting': Integer(), 'titlefounder': Integer(), 
       'locationName': String(100), 'companyName': String(100), 'description': String(400),'title': String(100),'industry': String(100)})

In [93]:
experience.to_sql('experience', engine, 'civictech', if_exists='replace', index=False)
# specify data types 

OperationalError: (pymysql.err.OperationalError) (1241, 'Operand should contain 1 column(s)')
[SQL: INSERT INTO civictech.experience (exp_id, ind_id, `locationName`, `companyName`, description, title, `startDate_month`, `startDate_year`, `endDate_month`, `endDate_year`, industry, company_empl_low, company_empl_high, title_direction, titleconsulting, titlefounder) VALUES (%(exp_id)s, %(ind_id)s, %(locationName)s, %(companyName)s, %(description)s, %(title)s, %(startDate_month)s, %(startDate_year)s, %(endDate_month)s, %(endDate_year)s, %(industry)s, %(company_empl_low)s, %(company_empl_high)s, %(title_direction)s, %(titleconsulting)s, %(titlefounder)s)]
[parameters: ({'exp_id': 0, 'ind_id': 0, 'locationName': 'Paris Metropolitan Region', 'companyName': 'bluenove', 'description': "Bluenove accompagne la transformation positive des organisations en faisant levier de l'intelligence collective de toutes leurs parties prenantes.\nPlus d'informations sur www.bluenove.com", 'title': 'Directeur a

### Education

In [25]:
education=pd.read_csv('profiles/people_education_db.csv', sep=';')

In [26]:
print(list(education.columns))
print(len(list(education.columns)))
print(len(education))

['ind_id', 'school', 'degreeName', 'schoolName', 'fieldOfStudy', 'startDate_year']
6
141


In [27]:
education.isna().sum().sort_values(ascending=False)

fieldOfStudy      51
school            46
degreeName        44
startDate_year    30
schoolName        19
ind_id             0
dtype: int64

In [28]:
education.dtypes.sort_values(ascending=False)

school             object
degreeName         object
schoolName         object
fieldOfStudy       object
startDate_year    float64
ind_id              int64
dtype: object

In [29]:
education.startDate_year.value_counts(dropna=False)
education['startDate_year'] = education['startDate_year'].fillna(0)

In [30]:
education['startDate_year']= education['startDate_year'].astype(int)

In [31]:
education.to_sql('education', engine, 'civictech', if_exists='replace', index=False)

141

# Organizations

## Companies finance data

In [47]:
companies_finance=pd.read_csv('scraping_organizations/companies_finance_data.csv', sep=';') # possible index drop

In [54]:
companies_finance.head()

,company,annee,chiffre_daffaires_e,marge_brute_e,resultat_dexploitation_e,resultat_net_e,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_operationnelle_pc,bfr_e,...,fonds_propres_e,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,citility,2017.0,0.0,0,0,-562000,0,0,0,-247000,...,1380000,0,0,0,0,0,0,0,0,0
1,citility,2016.0,30700.0,527000,-296000,-238000,4,1720,-965,-213000,...,382000,-775,-623,-173,351000,1140,636000,2070,9160,0
2,voxcracy,2019.0,46300.0,46300,-38500,-39000,177,100,-833,-44900,...,513000,-842,-76,-63,2170,47,33600,725,137,0
3,voxcracy,2018.0,16700.0,157000,-130000,-112000,861,940,-776,-27400,...,552000,-668,-202,-169,-21000,-126,101000,602,1120,5000
4,voxcracy,2017.0,1740.0,75900,-30300,-23400,0,4360,-1740,7680,...,569000,-1350,-41,-39,-5040,-290,19100,1100,329,0


In [52]:
companies_finance.drop(columns=['gestion_bfr', 'autonomie_financiere','solvabilite', 'rentabilite','structure_dactivite'], inplace=True)

In [56]:
companies_finance.reset_index(inplace=True)

In [58]:
companies_finance.rename(columns={"index": "company_year_ide"}, inplace=True)

In [60]:
companies_finance.head()

,company_year_ide,company,annee,chiffre_daffaires_e,marge_brute_e,resultat_dexploitation_e,resultat_net_e,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_operationnelle_pc,...,fonds_propres_e,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,0,citility,2017.0,0.0,0,0,-562000,0,0,0,...,1380000,0,0,0,0,0,0,0,0,0
1,1,citility,2016.0,30700.0,527000,-296000,-238000,4,1720,-965,...,382000,-775,-623,-173,351000,1140,636000,2070,9160,0
2,2,voxcracy,2019.0,46300.0,46300,-38500,-39000,177,100,-833,...,513000,-842,-76,-63,2170,47,33600,725,137,0
3,3,voxcracy,2018.0,16700.0,157000,-130000,-112000,861,940,-776,...,552000,-668,-202,-169,-21000,-126,101000,602,1120,5000
4,4,voxcracy,2017.0,1740.0,75900,-30300,-23400,0,4360,-1740,...,569000,-1350,-41,-39,-5040,-290,19100,1100,329,0


In [59]:
companies_finance.to_sql('companies_finance', engine, 'civictech', if_exists='replace', index=False)

74

### Companies info

In [63]:
companies_info=pd.read_csv('scraping_organizations/companies_info_data.csv', sep=';')
print(companies_info.columns)

Index(['company_id', 'companyName', 'Adresse :', 'Activité :', 'Effectif :',
       'Création :', 'Dirigeants :', 'Forme juridique :',
       'Inscription au RCS :', 'Capital social :',
       'Activité principale déclarée :', 'Code NAF ou APE :',
       'Domaine d’activité :', 'Dirigeant :', 'Inscription au RNA :',
       'Identifiant association :', 'Objet de l'association :',
       'Statut INSEE :'],
      dtype='object')


In [64]:
companies_info.head()

,company_id,companyName,Adresse :,Activité :,Effectif :,Création :,Dirigeants :,Forme juridique :,Inscription au RCS :,Capital social :,Activité principale déclarée :,Code NAF ou APE :,Domaine d’activité :,Dirigeant :,Inscription au RNA :,Identifiant association :,Objet de l'association :,Statut INSEE :
0,0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN


In [65]:
new_cols=[]
for i in list(companies_info.columns):
    i=i.lower().replace(' :', '').replace('é', 'e').replace("(", '').replace(')', '').replace(' ', '_').replace("'",'').replace("’",'').replace("è", "e")
    new_cols.append(i)
print(new_cols)

['company_id', 'companyname', 'adresse', 'activite', 'effectif', 'creation', 'dirigeants', 'forme_juridique', 'inscription_au_rcs', 'capital_social', 'activite_principale_declaree', 'code_naf_ou_ape', 'domaine_dactivite', 'dirigeant', 'inscription_au_rna', 'identifiant_association', 'objet_de_lassociation', 'statut_insee']


In [66]:
companies_info.columns=new_cols
companies_info.head()

,company_id,companyname,adresse,activite,effectif,creation,dirigeants,forme_juridique,inscription_au_rcs,capital_social,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK","SAS, société par actions simplifiée","INSCRIT (au greffe de LYON, le 12/05/2014)","39 620,00 €",Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN
1,1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,"SAS, société par actions simplifiée","INSCRIT (au greffe de MONTPELLIER, le 11/09/2015)","20 944,00 €",Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN
2,2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA","SAS, société par actions simplifiée","INSCRIT (au greffe de GRASSE, le 17/07/2014)","1 000,00 €","La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,NaN,NaN,NaN,NaN
3,3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 03/02/2015)","45 000,00 €",Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN
4,4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...","SAS, société par actions simplifiée","INSCRIT (au greffe de PARIS, le 21/11/2019)","56 445,00 €",L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN


In [67]:
companies_info['creation'] = pd.to_datetime(companies_info['creation'], format="%d/%m/%Y")

In [69]:
companies_info.to_sql('companies_info', engine, 'civictech', if_exists='replace', index=False)

76

In [70]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

# you can pass data types here 
# from sqlalchemy.types import Integer
# df.to_sql('integers', con=engine, index=False,
#           dtype={"A": Integer()})